# Building different tree-based models

In this activity we will build a random forest and a boosting model for the churn dataset. 
Let's first import the data.

## Dataset

We use a churn dataset again:

In [1]:
##### added line to ensure plots are showing
%matplotlib inline
#####

import pandas as pd
import numpy as np

df = pd.read_csv('churn_ibm.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


You already know the pre-processing steps from before:

In [2]:
y = df['Churn']
X = df.drop(['Churn','customerID'],axis=1)

for column in X.columns:
    if X[column].dtype == object:
        X = pd.concat([X,pd.get_dummies(X[column], prefix=column, drop_first=True)],axis=1).drop([column],axis=1)
        
y = pd.get_dummies(y, prefix='churn', drop_first=True)

## Random forest

We can very easily calculate a random forest:

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

rf = RandomForestClassifier(n_estimators=20)

# Some algorithms need a transformed version of the dependent variable
# To this purpose, the data is reshaped using ravel()
rf.fit(X_train,y_train.values.ravel())
prediction = rf.predict(X_test)
print('Accuracy:', accuracy_score(y_test,prediction))

Accuracy: 0.771090047393365


We can print the instance of ```RandomForestClassifier``` to learn about the default settings used:

In [9]:
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 20, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


We can also change the parameters. Let's build a larger forest using more trees using parameter ```n_estimators```:

In [13]:
rf2 = RandomForestClassifier(n_estimators=100)
rf2.fit(X_train,y_train.values.ravel())
prediction = rf2.predict(X_test)

print('Accuracy:', accuracy_score(y_test,prediction))

Accuracy: 0.7834123222748816


Notice what is contained in the prediction array, it's all binary:

In [14]:
print('Prediction:',prediction[0:10])

Prediction: [0 0 0 0 1 0 0 0 0 0]


We can also output the probabilities for every class, as follows, and use it for AUC:

In [15]:
prediction_prob = rf2.predict_proba(X_test)
print('Prediction:',prediction_prob[0:10])
print('AUC:',roc_auc_score(y_test,prediction_prob[:,1]))

Prediction: [[0.9  0.1 ]
 [0.56 0.44]
 [0.97 0.03]
 [0.94 0.06]
 [0.49 0.51]
 [0.61 0.39]
 [0.68 0.32]
 [0.87 0.13]
 [0.93 0.07]
 [0.68 0.32]]
AUC: 0.8160878633000935


## Boosting

sklearn allows us to implement alternative versions of the boosting method (mainly AdaBoost and Gradient Boosting). Let's impllement the AdaBoost.

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train,y_train.values.ravel())
prediction = ada.predict(X_test)
prediction_prob = ada.predict_proba(X_test)
print('Accuracy:', accuracy_score(y_test,prediction))
print('AUC:',roc_auc_score(y_test,prediction_prob[:,1]))

Accuracy: 0.7971563981042654
AUC: 0.8371850781922725


Apparently, 50 trees are used:

In [18]:
print(ada.get_params())

{'algorithm': 'SAMME.R', 'base_estimator': None, 'learning_rate': 1.0, 'n_estimators': 50, 'random_state': None}


Let's increase that as well:

In [19]:
ada2 = AdaBoostClassifier(n_estimators=100)
ada2.fit(X_train,y_train.values.ravel())
prediction = ada2.predict(X_test)
prediction_prob = ada.predict_proba(X_test)
print('Accuracy:', accuracy_score(y_test,prediction))
print('AUC:',roc_auc_score(y_test,prediction_prob[:,1]))

Accuracy: 0.7895734597156399
AUC: 0.8371850781922725


Neither accuracy, nor AUC are improved drastically, but AdaBoost performs better than the random forest we built.

## Grid search

A lot of these efforts can be streamlined using GridSearch. Below, you can find code that tests different parameters using cross-validation for random forests:

In [20]:
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import GridSearchCV

# First we create a dictionary containing the parameters we want to test
# We include the values we want to test as lists 
parameters = {'min_samples_leaf':[1,5],'max_depth':[None,10]}

# Then, we bring together a classifier, the parameters, and set the number of folds for the CV
grid_search = GridSearchCV(RandomForestClassifier(n_estimators=20), parameters, cv=10)
grid_search.fit(X_train, y_train.values.ravel())

# The best predictor will be used for the prediction
prediction = grid_search.predict(X_test)
    
best_classifier = grid_search.best_estimator_

print('Best classifier:',best_classifier)
print('Accuracy:', accuracy_score(y_test,prediction))
print('AUC:',roc_auc_score(y_test,prediction))

Best classifier: RandomForestClassifier(max_depth=10, min_samples_leaf=5, n_estimators=20)
Accuracy: 0.7924170616113744
AUC: 0.6950547669972131


It seems that having a minimum of 5 samples per leaf, and a maximum depth of 10 are preferable.

## Feature importance

For all our models, we can calculate the feature importance. This is the (average) reduction in Gini impurity across all trees:

In [21]:
# Random forest - 5 most important features
for c, column in enumerate(X_test.columns):
    if rf.feature_importances_[c] in sorted(rf.feature_importances_)[-5:]:
        print('Variable',column,rf.feature_importances_[c])

Variable tenure 0.2049328162223294
Variable MonthlyCharges 0.1679798883435498
Variable TotalCharges 0.17426998824541678
Variable InternetService_Fiber optic 0.039967830939111396
Variable PaymentMethod_Electronic check 0.027061388685309485


In [22]:
# AdaBoost - 5 most important features
for c, column in enumerate(X_test.columns):
    if ada.feature_importances_[c] in sorted(ada.feature_importances_)[-5:]:
        print('Variable',column,ada.feature_importances_[c])

Variable tenure 0.16
Variable MonthlyCharges 0.2
Variable TotalCharges 0.26
Variable MultipleLines_No phone service 0.04
Variable InternetService_Fiber optic 0.06
Variable Contract_Two year 0.04


It seems that both random forests and AdaBoost have exactly the same variables driving their Gini impurity down. Mostly the length of tenure, monthly charges, total charges, being connected through fiber and the contract length.